# 104人力銀行職缺爬蟲

### 匯入所有所需套件

In [55]:
import pandas as pd
import json
import re
import requests
import time
from time import sleep
import csv
import random
from bs4 import BeautifulSoup as bs

## 爬取單一分頁內容

### 設定欲爬取之網址及內容

In [56]:
search_url = "https://www.104.com.tw/jobs/search/?cat=2007000000&jobsource=2018indexpoc&ro=0"
head = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
        'Accept-Language':'zh-TW,zh;q=0.8,en-US;q=0.6,en;q=0.4'} #模擬真人查詢
# 使用 beautifulsoup4解析來自requests的search_url文檔
res = requests.get(search_url, headers = head)
print(res.encoding)
res.encoding = 'utf8'
soup = bs(res.text)

UTF-8


In [57]:
#搜尋所有職缺
jobs = soup.find_all('article',class_= 'js-job-item')
jobs

[<article class="b-block--top-bord job-list-item b-clearfix js-job-item js-job-item--focus b-block--ad" data-cust-name="新加坡商耐思人工智能有限公司台灣分公司" data-cust-no="130000000143942" data-indcat="" data-indcat-desc="" data-is-apply="0" data-is-save="0" data-job-name="Frontend Engineer / 前端工程師 (React)" data-job-no="11775501" data-job-ro="1" data-jobsource="hotjob_chr" data-qa-id="jobSeachResult">
 <div class="b-block__left">
 <h2 class="b-tit">
 <span class="b-tit__date"> <svg class="b-icon--gray b-icon--w18">
 <use xlink:href="//www.104.com.tw/jobs/search/static/img/sprite.svg#icon-focus"></use>
 </svg> </span>
 <a class="js-job-link" data-qa-id="jobSeachResultTitle" href="//www.104.com.tw/job/70e19?jobsource=hotjob_chr" target="_blank">Frontend Engineer / 前端工程師 (React)</a>
 </h2>
 <ul class="b-list-inline b-clearfix">
 <li class="b-list-inline__ad-icon"> </li>
 <li>
 <a href="//www.104.com.tw/company/1a2x6bl2di?jobsource=hotjob_chr" target="_blank" title="公司名：新加坡商耐思人工智能有限公司台灣分公司
 公司住址：台北市內湖區瑞湖街1

In [58]:
for job in jobs:
    job_name = job.find('a',class_='js-job-link').text
    job_company = job.get('data-cust-name')
    job_loc = job.find('ul',class_='job-list-intro').find('li').text
    job_pay = job.find('span',class_='b-tag--default').text
    job_url = job.find("a").get('href')
    
    print(" 職缺內容:",job_name,"\n","公司名稱:",job_company,"\n","地址:",job_loc,"\n","薪資:",job_pay,"\n","網址:",job_url,"\n",'-'*70)
time.sleep(random.randint(1,3))

 職缺內容: Frontend Engineer / 前端工程師 (React) 
 公司名稱: 新加坡商耐思人工智能有限公司台灣分公司 
 地址: 台北市內湖區 
 薪資: 待遇面議 
 網址: //www.104.com.tw/job/70e19?jobsource=hotjob_chr 
 ----------------------------------------------------------------------
 職缺內容: 電機工程師 
 公司名稱: 裕發科技股份有限公司 
 地址: 台中市烏日區 
 薪資: 月薪45,000~70,000元 
 網址: //www.104.com.tw/job/6v6ef?jobsource=hotjob_chr 
 ----------------------------------------------------------------------
 職缺內容: MIS資訊工程師 Supervisor / Sr. Supervisor 
 公司名稱: 英屬維京群島商華敦國際有限公司台灣分公司 
 地址: 台北市松山區 
 薪資: 待遇面議 
 網址: //www.104.com.tw/job/7grit?jobsource=hotjob_chr 
 ----------------------------------------------------------------------
 職缺內容: .NET 工程師(Junior/Senior) 
 公司名稱: 全支付電子支付股份有限公司 
 地址: 台北市中山區 
 薪資: 年薪560,000~1,300,000元 
 網址: //www.104.com.tw/job/7996x?jobsource=hotjob_chr 
 ----------------------------------------------------------------------
 職缺內容: CRD 手機遊戲工程師【歡迎應屆畢業生】 
 公司名稱: 慧邦科技股份有限公司 
 地址: 台北市中正區 
 薪資: 待遇面議 
 網址: //www.104.com.tw/job/4xvm4?jobsource=hotjob_chr 
 --------------

### 資料存成csv檔
#### 兩種方法都不用先在資料夾中設好檔案位置

### 方法一: 透過pandas套件儲存

In [59]:
content=[]

for job in jobs:
    job_name = job.find('a',class_='js-job-link').text
    job_company = job.get('data-cust-name')
    job_loc = job.find('ul',class_='job-list-intro').find('li').text
    job_pay = job.find('span',class_='b-tag--default').text
    job_url = job.find("a").get('href')
    job_data = {"職缺內容":job_name,"公司名稱":job_company,"地址":job_loc,"薪資":job_pay,"網址":job_url}
    content.append(job_data)
# 資料輸出
df = pd.DataFrame(content)
df.to_csv('104職缺內容(方法一).csv',encoding='big5')

### 方法二: 透過csv套件儲存

In [60]:
fn = "104職缺內容.csv" #取csv檔名
columns_name = ["職缺內容","公司名稱","地址","薪資","網址"] #第一欄的名稱
with open(fn,"w",newline="",) as csvFile: #定義csv的寫入檔，每次寫完會換行
    dictWriter = csv.DictWriter(csvFile,fieldnames=columns_name) #定義寫入器
    dictWriter.writeheader()
    for job in jobs:
        job_name = job.find('a',class_='js-job-link').text
        job_company = job.get('data-cust-name')
        job_loc = job.find('ul',class_='job-list-intro').find('li').text
        job_pay = job.find('span',class_='b-tag--default').text
        job_url = job.find("a").get('href')
        job_data = {"職缺內容":job_name,"公司名稱":job_company,"地址":job_loc,"薪資":job_pay,"網址":job_url}
        dictWriter.writerow(job_data)

### 資料存成json檔

In [61]:
fn = "104職缺內容.json"
obj = {}
obj["104職缺內容"] = []
for job in jobs:
    job_name = job.find('a',class_='js-job-link').text
    job_company = job.get('data-cust-name')
    job_loc = job.find('ul',class_='job-list-intro').find('li').text
    job_pay = job.find('span',class_='b-tag--default').text
    job_url = job.find("a").get('href')
    name = {"職缺內容": job_name,"公司名稱": job_company,"地址": job_loc,"薪資": job_pay,"網址": job_url}
    obj["104職缺內容"].append(name)
with open(fn, 'w', encoding='utf-8') as json_obj:
    json.dump(obj, json_obj,ensure_ascii=False)

## 爬取多個分頁內容 (104為AJAX動態載入網頁)
### 還在嘗試中@_@

### url_1+頁數+url_2=搜尋結果(link)

In [97]:
url_1 = "https://www.104.com.tw/jobs/search/?ro=0&jobcat=2007000000&expansionType=area%2Cspec%2Ccom%2Cjob%2Cwf%2Cwktm&order=17&asc=0&page="
url_2 = "&mode=s&jobsource=2018indexpoc&langFlag=0&langStatus=0&recommendJob=1&hotJob=1"

all_job_datas=[]
for page in range(1,5+1):
    url = url_1+str(page)+url_2
    print(url)
    head = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
        'Accept-Language':'zh-TW,zh;q=0.8,en-US;q=0.6,en;q=0.4'} 
    htmlFile = requests.get(url, headers = head)
    soups = bs(res.text)
    manyjobs = soups.find_all('article',class_= 'js-job-item')
    for job in manyjobs:
        job_name = job.find('a',class_='js-job-link').text
        job_company = job.get('data-cust-name')
        job_loc = job.find('ul',class_='job-list-intro').find('li').text
        job_pay = job.find('span',class_='b-tag--default').text
        job_url = job.find("a").get('href')
        job_data = {"職缺內容":job_name,"公司名稱":job_company,"地址":job_loc,"薪資":job_pay,"網址":job_url}
        all_job_datas.append(job_data)
        print(job_data)
    time.sleep(random.randint(1,3))

https://www.104.com.tw/jobs/search/?ro=0&jobcat=2007000000&expansionType=area%2Cspec%2Ccom%2Cjob%2Cwf%2Cwktm&order=17&asc=0&page=1&mode=s&jobsource=2018indexpoc&langFlag=0&langStatus=0&recommendJob=1&hotJob=1
{'職缺內容': '系統工程師(台北總公司)', '公司名稱': '聯華食品工業股份有限公司', '地址': '台北市大同區', '薪資': '月薪38,000元以上', '網址': '//www.104.com.tw/job/7hk72?jobsource=jolist_b_date'}
{'職缺內容': 'Vue.js 前端工程師', '公司名稱': '尚程資訊有限公司', '地址': '台北市信義區', '薪資': '待遇面議', '網址': '//www.104.com.tw/job/7ona5?jobsource=jolist_b_date'}
{'職缺內容': 'Python軟體開發工程師(新竹)', '公司名稱': '友宏整合行銷有限公司', '地址': '新竹市', '薪資': '月薪36,000~80,000元', '網址': '//www.104.com.tw/job/72iao?jobsource=jolist_b_date'}
{'職缺內容': '資安軟體開發工程師(台北)', '公司名稱': '友宏整合行銷有限公司', '地址': '台北市南港區', '薪資': '月薪35,000~80,000元', '網址': '//www.104.com.tw/job/72tds?jobsource=jolist_b_date'}
{'職缺內容': '網站後端工程師', '公司名稱': '大禾商社股份有限公司', '地址': '台南市東區', '薪資': '月薪30,000~60,000元', '網址': '//www.104.com.tw/job/6wuyh?jobsource=jolist_b_date'}
{'職缺內容': '網路工程師(HPE/Aruba/Cisco) -台北', '公司名稱': '擎昊科技股份有限公司', '地址':

https://www.104.com.tw/jobs/search/?ro=0&jobcat=2007000000&expansionType=area%2Cspec%2Ccom%2Cjob%2Cwf%2Cwktm&order=17&asc=0&page=4&mode=s&jobsource=2018indexpoc&langFlag=0&langStatus=0&recommendJob=1&hotJob=1
{'職缺內容': '系統工程師(台北總公司)', '公司名稱': '聯華食品工業股份有限公司', '地址': '台北市大同區', '薪資': '月薪38,000元以上', '網址': '//www.104.com.tw/job/7hk72?jobsource=jolist_b_date'}
{'職缺內容': 'Vue.js 前端工程師', '公司名稱': '尚程資訊有限公司', '地址': '台北市信義區', '薪資': '待遇面議', '網址': '//www.104.com.tw/job/7ona5?jobsource=jolist_b_date'}
{'職缺內容': 'Python軟體開發工程師(新竹)', '公司名稱': '友宏整合行銷有限公司', '地址': '新竹市', '薪資': '月薪36,000~80,000元', '網址': '//www.104.com.tw/job/72iao?jobsource=jolist_b_date'}
{'職缺內容': '資安軟體開發工程師(台北)', '公司名稱': '友宏整合行銷有限公司', '地址': '台北市南港區', '薪資': '月薪35,000~80,000元', '網址': '//www.104.com.tw/job/72tds?jobsource=jolist_b_date'}
{'職缺內容': '網站後端工程師', '公司名稱': '大禾商社股份有限公司', '地址': '台南市東區', '薪資': '月薪30,000~60,000元', '網址': '//www.104.com.tw/job/6wuyh?jobsource=jolist_b_date'}
{'職缺內容': '網路工程師(HPE/Aruba/Cisco) -台北', '公司名稱': '擎昊科技股份有限公司', '地址':

### 資料存成csv檔

In [94]:
fn = "104職缺內容(多頁).csv"
columns_name = ["職缺內容","公司名稱","地址","薪資","網址"]
with open(fn,"w",newline="",encoding="utf_8_sig") as csvFile:
    dictWriter = csv.DictWriter(csvFile,fieldnames=columns_name,)
    dictWriter.writeheader()
    for data in all_job_datas:
        dictWriter.writerow(data)

### 資料存成json檔

In [95]:
fn = "104職缺內容(多頁).json"
obj = {}
obj["104職缺內容"] = []
for data in all_job_datas:
    name = {"職缺內容": job_name,"公司名稱": job_company,"地址": job_loc,"薪資": job_pay,"網址": job_url}
    obj["104職缺內容"].append(name)
with open(fn, 'w', encoding='utf-8') as json_obj:
    json.dump(obj, json_obj,ensure_ascii=False)